In [ ]:
import seaborn as sns
sns.set()

In [ ]:
from static_grader import grader

# Spark Miniproject


Stack Overflow is a collaboratively edited question-and-answer site originally focused on programming topics. Because of the variety of features tracked, including a variety of feedback metrics, it allows for some open-ended analysis of user behavior on the site.

Stack Exchange (the parent organization) provides an anonymized [data dump](https://archive.org/details/stackexchange), and we'll use Spark to perform data manipulation, analysis, and machine learning on this data set. As a side note, there's also an online data explorer which allows you to query the data interactively.

*Consider*: Do we need to use Spark to work with this data set? What are our alternatives?

## Workflow


Most questions can be done locally, however in some cases you may want to use cloud services. See the appropriate lecture notebooks for information on how to use cloud services.

Python example:

1. Edit source code in your `main.py` file, classes in a separate `classes.py` (class definitions need to be written in a separate file and then included at runtime).
1. Run locally on a chunk using eg. `$SPARK_HOME/bin/spark-submit --py-files src/classes.py src/main.py data/stats results/stats/`
1. Run on GCP once your testing and development are done.

General tips:
* Try `cat output_dir/* | sort -n -t , -k 1.2 -o sorted_output` to concatenate your output files, which will also be in `part-xxxxx` format.
* You can access an interactive PySpark shell with `$SPARK_HOME/bin/pyspark`.

## Accessing the data


The data is available on S3 (`s3://dataincubator-course/spark-stack-data`). There are three sub-folders, `allUsers`, `allPosts`, and `allVotes`, which contain Gzipped XML with the following format:

``` html
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

Data from the much smaller `stats.stackexchange.com` is available in the same format on S3 (`s3://dataincubator-course/spark-stats-data`). This site, Cross-Validated, will be used below in some instances to avoid working with the full data set for every question.

The full schema is available as a text file:

In [ ]:
!aws s3 cp s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt .

You can either get the data by running the appropriate S3 commands in the terminal, or by running this block for the smaller stats data set:

In [ ]:
!mkdir -p spark-stats-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data
!aws s3 sync --exclude '*' --include 'posts*zip' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data

And to get the much larger full data set (be warned, this can take 20 or more minutes, so you may want to run it in the terminal to avoid locking up the notebook):

In [ ]:
!mkdir -p spark-stack-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stack-data/ ./spark-stack-data

## Data input and parsing


Some rows are split across multiple lines; these can be discarded. Incorrectly formatted XML can also be ignored. It is enough to simply skip problematic rows, the loss of data will not significantly impact our results on this large data sets.

You will need to handle XML parsing yourself.  Our solution uses `lxml.etree` in Python.

To make your code more flexible, it's also recommended to incorporate command-line arguments that specify the location of the input data and where output should be written.

The goal should be to have a parsing function that can be applied to the input data to access any XML element desired. It is suggested to use a class structure so that you can create RDDs of Posts, Votes, Users, etc.

``` python
# Command line arguments using sysv or argparse in Python
if __name__ == '__main__':
    main(ARGS.input_dir, ARGS.output_dir)
```

## Questions

## Bad XML


A simple question to test your parsing code. Create an RDD of Post objects where each Post is a valid row of XML from the Cross-Validated (stats.stackexchange.com) `allPosts` data set.

We are going to take several shortcuts to speed up and simplify our computations.  First, your parsing function should only attempt to parse rows that start with `<row` as these denote actual data entries. This should be done in Spark as the data is being read in from disk, without any pre-Spark processing. 

Return the total number of XML rows that started with `<row` that were subsequently **rejected** during your processing.  Note that the text is unicode, and contains non-ASCII characters.  You may need to re-encode to UTF-8 (depending on your XML parser)

Note that this cleaned data set will be used for all subsequent questions.

*Question*: Can you figure out what filters you need to put in place to avoid throwing parsing errors entirely?

In [ ]:
from pyspark import SparkContext

sc = SparkContext("local[*]", "temp")

import os, time
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

In [ ]:
posts_xml = sc.textFile(localpath('spark-stats-data/allPosts/*'))

In [ ]:
votes_xml = sc.textFile(localpath('spark-stats-data/allVotes/*'))

In [ ]:
users_xml = sc.textFile(localpath('spark-stack-data/allUsers/*'))

In [ ]:
from lxml import etree

def xmlparser(xmlstring):
    try:
        root = etree.fromstring(xmlstring.encode('utf-8'), etree.XMLParser(encoding='utf-8'))
        return root.attrib
    except:
        return None

In [ ]:
def isRow(line):
    return "<row" in line

In [ ]:
class recordPost(object):
    def __init__(self, tags, userID, date, typeID, score, views, answers, favorites, answerID):
        self.tags = tags
        self.userID = userID
        self.date = date
        self.typeID = typeID
        self.score = score
        self.views = views
        self.answers = answers
        self.favorites = favorites
        self.answerID = answerID

    @classmethod
    def parse(cls, line):
        attributes = xmlparser(line)
        tags = attributes.get('Tags')
        userID = attributes.get('OwnerUserId')
        date = attributes.get('CreationDate')
        typeID = attributes.get('PostTypeId')
        score = int(attributes.get('Score', "0"))
        views = int(attributes.get('ViewCount', "0"))
        answers = int(attributes.get('AnswerCount', "0"))
        favorites = int(attributes.get('FavoriteCount', "0"))
        answerID = attributes.get('AcceptedAnswerId')
        return cls(tags, userID, date, typeID, score, views, answers, favorites, answerID)
    
class recordUser(object):
    def __init__(self, ID, date, reputation):
        self.ID = ID
        self.date = date
        self.reputation = reputation

    @classmethod
    def parse(cls, line):
        attributes = xmlparser(line)
        ID = attributes.get('Id')
        date = attributes.get('CreationDate')
        reputation = attributes.get('Reputation')
        return cls(ID, date, reputation)
    
class recordVote(object):
    def __init__(self, typeID, postID):
        self.typeID = typeID
        self.postID = postID

    @classmethod
    def parse(cls, line):
        attributes = xmlparser(line)
        typeID = attributes.get('VoteTypeId')
        postID = attributes.get('PostId')
        return cls(typeID, postID)
    
class Record(object):
    def __init__(self, attributes):
        self.attr = attributes

    @classmethod
    def parse(cls, line):
        attributes = xmlparser(line)
        return cls(attributes)

In [ ]:
post_record = posts_xml.filter(lambda x: isRow(x)) \
                .filter(lambda x: xmlparser(x) is not None) \
                .map(recordPost.parse)
    
vote_record = votes_xml.filter(lambda x: isRow(x)) \
                .filter(lambda x: xmlparser(x) is not None) \
                .map(recordVote.parse)
    
user_record = users_xml.filter(lambda x: isRow(x)) \
                .filter(lambda x: xmlparser(x) is not None) \
                .map(recordUser.parse)

In [ ]:
badXMLcount = posts_xml.filter(lambda x: isRow(x)) \
                    .filter(lambda x: xmlparser(x) is None) \
                    .count()

In [ ]:
raw = sc.textFile(localpath('spark-stats-data/allPosts'))

In [ ]:
xmls=raw.filter(lambda x: '<row' in x)

In [ ]:
xmls.count()


In [ ]:
from lxml import etree
a=etree.fromstring('''<row Body="" CommentCount="0" CreationDate="2013-10-28T10:42:29.940" Id="73933" LastActivityDate="2013-10-28T10:42:29.940" LastEditDate="2013-10-28T10:42:29.940" LastEditorUserId="686" OwnerUserId="686" PostTypeId="5" Score="0" />''')

In [ ]:
a.getchildren()
'Score' in a.attrib

In [ ]:
#XMLsyntaxError
import xml.etree.ElementTree as ET

In [ ]:
def parsePost(line):
    try:
        root = ET.fromstring(line)
        return line
    except:
        return 0

In [ ]:
posts = xmls.map(parsePost).filter(lambda x: x!= 0)

In [ ]:
posts.count()

## Favorites and scores

We're interested in looking for useful patterns in the data.  If we look at the Post data again (the smaller set, `stats.stackexchange.com`), we see that many things about each post are recorded.  We're going to start by looking to see if there is a relationship between the number of times a post was favorited (the `FavoriteCount`) and the `Score`.  The score is the number of times the post was upvoted minus the number of times it was downvoted, so it is a measure of how much a post was liked.  We'd expect posts with a higher number of favorites to have better scores, since they're both measurements of how good the post is.

Let's aggregate posts by the number of favorites, and find the average score for each number of favorites.  Do this for the lowest 50 numbers of favorites.

_Note:_ Before submitting, take a look at the numbers.  Do they follow the trend you expect?

**Checkpoints**

- Total score across all posts: 299469
- Mean of first 50 favorite counts (averaging the keys themselves): 24.76

In [ ]:
def fav(line):
    try:
        root = ET.fromstring(line)
        if "Score" and "FavoriteCount" in root.attrib:
            score = root.attrib['Score']
            fav = root.attrib['FavoriteCount']
            return (int(fav),int(score))
        else:
            return ('Empty')
    except:
        return ('Empty')

In [ ]:
favs=posts.map(fav).filter(lambda x: x != 'Empty')

In [ ]:
favs.sortByKey()\
    .reduceByKey(lambda x,y: x+y).take(5)

In [ ]:
favs.sortByKey(lambda x: x[0])\
    .reduceByKey(lambda x,y: x+y).count()

In [ ]:
favs.map(lambda x: x[0]).map(lambda x: (x,1))\
    .sortByKey(lambda x: x[0])\
    .reduceByKey(lambda x,y: x+y).take(5)

In [ ]:
favs.map(lambda x: x[0]).map(lambda x: (x,1))\
    .reduceByKey(lambda x,y: x+y).count()

In [ ]:
total = favs.sortByKey(lambda x: x[0])\
            .reduceByKey(lambda x,y: x+y)

count = favs.map(lambda x: x[0]).map(lambda x: (x,1))\
            .sortByKey(lambda x: x[0])\
            .reduceByKey(lambda x,y: x+y)

In [ ]:
total.join(count).map(lambda x: (x[0],x[1][0]/x[1][1])).collect()

In [ ]:
#favorite_score = [(0, 2.3398827696988396)]*50
favorite = list(total.join(count).map(lambda x: (x[0],x[1][0]/x[1][1])).collect())

In [ ]:
favorite_score = sorted(favorite, key=lambda x: x[0])[:50]

In [ ]:
# favorite_score = [(0, 2.3398827696988396)]*50
grader.score('spark__favorite_score', favorite_score)

## Answer percentage


Investigate the correlation between a user's reputation and the kind of posts they make. For the 99 users with the highest reputation, single out posts which are either questions or answers and look at the percentage of these posts that are answers: *(answers / (answers + questions))*. 

Return a tuple of their **user ID** and this fraction.

You should also return (-1, fraction) to represent the case where you average over all users (so you will return 100 entries total).

Again, you only need to run this on the statistics overflow set.


#### Checkpoints

* Total questions: 52,060
* Total answers: 55,304
* Top 99 users' average reputation: 11893.464646464647

In [ ]:
users = sc.textFile(localpath('spark-stats-data/allUsers'))

In [ ]:
##get reputation score
def rep(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line)
            if "Id" and "Reputation" in root.attrib:
                uid = root.attrib['Id']
                rep = root.attrib['Reputation']
                return (uid,int(rep))
            else:
                return ('Empty')
        except:
            return ('Empty')
    else:
        return ('Empty')
    
all_users=users.map(rep).filter(lambda x: x != 'Empty').sortBy(lambda x: -x[1])

##get post with questions and answers
def post(line):
    try:
        root = ET.fromstring(line)
        if "PostTypeId" and "OwnerUserId" in root.attrib:
            uid = root.attrib['OwnerUserId']
            ptype = int(root.attrib['PostTypeId'])
            if ptype == 1 or ptype == 2:
                return (uid,ptype)
            else:
                return ('Empty')
        else:
            return ('Empty')
    except:
        return ('Empty')
    
all_p = posts.map(post).filter(lambda x: x != 'Empty')

questions = all_p.filter(lambda x: x[1] == 1).reduceByKey(lambda x,y: x+y)

answers = all_p.filter(lambda x: x[1] == 2).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0],int(x[1]/2)))

##merge 
answer_percentage_long=all_users.join(questions).join(answers)\
                                .map(lambda x: (int(x[0]),x[1][0][0],x[1][0][1],x[1][1]))\
                                .map(lambda x: (x[0],x[1],x[3]/(x[2]+x[3])))\
                                .takeOrdered(99, key=lambda x: -x[1])

answer_percentage=[(x[0],x[2]) for x in answer_percentage_long]

##calculate average
total, count = all_users.join(questions).join(answers).map(lambda x: (int(x[0]),x[1][0][0],x[1][0][1],x[1][1]))\
                        .map(lambda x: (x[0],x[3]/(x[2]+x[3]),1))\
                        .map(lambda x: (x[1],x[2]))\
                        .reduce(lambda x, y: (x[0]+y[0], x[1]+y[1]))

avg = (-1,total/count)
answer_percentage.append(avg)

In [ ]:
# answer_percentage = [(7071, 0.9107142857142857)] * 100

grader.score('spark__answer_percentage', answer_percentage)

## First question

We'd expect the first **question** a user asks to be indicative of their future behavior.  We'll dig more into that in the next problem, but for now let's see the relationship between reputation and how long it took each person to ask their first question.

For each user that asked a question, find the difference between when their account was created (`CreationDate` for the User) and when they asked their first question (`CreationDate` for their first question).  Return this time difference in days (round down, so 2.7 days counts as 2 days) for the 100 users with the highest reputation, in the form

`(UserId, Days)`

**Checkpoints**
- Users that asked a question: 23134
- Average number of days (round each user's days, then average): 30.1074258

In [ ]:
def first_question(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line)
            if "Id" and "Reputation" and 'CreationDate' in root.attrib:
                uid = root.attrib['Id']
                rep = root.attrib['Reputation']
                cdate = root.attrib['CreationDate']
                return (int(uid),int(rep),cdate)
            else:
                return ('Empty')
        except:
            return ('Empty')
    else:
        return ('Empty')

In [ ]:
from datetime import timedelta, datetime
def parsetime(x):
    if x is None:
        return None
    else:
        return datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%f')

In [ ]:
user_info = users.map(first_question).filter(lambda x: x != 'Empty')\
                 .map(lambda x: (x[0], (x[1], parsetime(x[2]))))

In [ ]:
def post_first_question(line):
    try:
        root = ET.fromstring(line)
        if "PostTypeId" and "OwnerUserId" and 'CreationDate' in root.attrib:
            uid = root.attrib['OwnerUserId']
            ptype = int(root.attrib['PostTypeId'])
            cdate = root.attrib['CreationDate']
            if ptype == 1:
                return (int(uid),cdate)
            else:
                return ('Empty')
        else:
            return ('Empty')
    except:
        return ('Empty')

In [ ]:
post_info = posts.map(post_first_question).filter(lambda x: x != 'Empty')\
                 .map(lambda x: (x[0], parsetime(x[1])))\
                 .reduceByKey(lambda x, y: min(x, y))

In [ ]:
user_info.take(5)

In [ ]:
def time_delta(x,y):
    if x is None or y is None:
        return None
    else:
        return (x-y).days

In [ ]:
first_questions = user_info.join(post_info).map(lambda x: (x[0],x[1][0][0],x[1][0][1],x[1][1]))\
.map(lambda x:(x[0],x[1],time_delta(x[3],x[2]))).takeOrdered(100, key=lambda x: -x[1])

In [ ]:
first_question=[(x[0],x[2]) for x in first_questions]

In [ ]:
# first_question = [(805, 669)] * 100

grader.score('spark__first_question', first_question)

## Identify veterans


It can be interesting to think about what factors influence a user to remain active on the site over a long period of time. In order not to bias the results towards older users, we'll define a time window between 100 and 150 days after account creation. If the user has made a post in this time, we'll consider them active and well on their way to being veterans of the site; if not, they are inactive and were likely brief users.

*Consider*: What other parameterizations of "activity" could we use, and how would they differ in terms of splitting our user base?

*Consider*: What other biases are still not dealt with, after using the above approach?

Let's see if there are differences between the first ever question posts of "veterans" vs. "brief users". For each group separately, average the score, views, number of answers, and number of favorites of the users' **first question**.

*Consider*: What story could you tell from these numbers? How do the numbers support it?


In [ ]:
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

posts = sc.textFile(localpath('spark-stats-data/allPosts'))
users = sc.textFile(localpath('spark-stats-data/allUsers'))

#define functions
def parsetime(x):
    if x is None:
        return None
    else:
        return datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%f')
    
def time_delta(x,y):
    if x is None or y is None:
        return None
    else:
        return (x-y).days
    
def user_parse(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line)
            if "Id" in root.attrib:
                uid = root.attrib['Id']
                cdate = root.attrib.get('CreationDate',None)
                return (int(uid),cdate)
            else:
                return ('Empty')
        except:
            return ('Empty')
    else:
        return ('Empty')
    
def date_post(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line)
            if "OwnerUserId" in root.attrib:
                uid = root.attrib['OwnerUserId']
                cdate = root.attrib.get('CreationDate',None)
                return (int(uid),cdate)
            else:
                return ('Empty')
        except:
            return ('Empty')
    else:
        return ('Empty')
    
def vet(x):
    if x is None:
        return 0
    if int(x) >= 100 and int(x) <= 150:
        return 1
    else:
        return 0

##get rdds
valid_users = users.map(user_parse).filter(lambda x: x != 'Empty').map(lambda x: (x[0],parsetime(x[1])))    

valid_posts = posts.map(date_post).filter(lambda x: x != 'Empty')\
                        .map(lambda x: (x[0],parsetime(x[1])))

all_user_info = valid_users.join(valid_posts)\
                           .map(lambda x: (x[0],time_delta(x[1][1],x[1][0])))         

all_i = all_user_info.map(lambda x: (x[0],vet(x[1]))).reduceByKey(lambda x,y: max(x,y))

veteran = all_i.filter(lambda x: x[1] == 1)
brief_views = all_i.filter(lambda x: x[1] == 0)

def post_first_question_more(line):
    try:
        root = ET.fromstring(line)
        if "PostTypeId" and "OwnerUserId" and 'CreationDate' in root.attrib:
            uid = root.attrib['OwnerUserId']
            ptype = int(root.attrib['PostTypeId'])
            cdate = root.attrib['CreationDate']
            #a.attrib.get('Score',1)
            answer = root.attrib.get("AnswerCount",0)           
            view = root.attrib.get("ViewCount",0)           
            fav = root.attrib.get("FavoriteCount",0)            
            s = root.attrib.get("Score",0) 
            #filter out questions
            if ptype == 1:
                return (int(uid),cdate,int(answer),int(view),int(fav),int(s))
            else:
                return ('Empty')
        else:
            return ('Empty')
    except:
        return ('Empty')

#find out the first questions date
first_q= posts_long.map(post_first_question_more).filter(lambda x: x != 'Empty')\
              .map(lambda x: (x[0], parsetime(x[1])))\
              .reduceByKey(lambda x, y: min(x, y))\
              .map(lambda x: ((x[0],x[1]),1))

#find out all questions' performance
all_q = posts_long.map(post_first_question_more).filter(lambda x: x != 'Empty')\
              .map(lambda x: ((x[0],parsetime(x[1])),(x[2],x[3],x[4],x[5])))

#merge (acount, first_question_date) with performances
stats = first_q.join(all_q).map(lambda x: (x[0][0],(x[1][1][0],x[1][1][1],x[1][1][2],x[1][1][3])))

##calculate average for veterian
a,v,f,s,t=veteran.join(stats).map(lambda x: (x[0],(x[1][1][0],x[1][1][1],x[1][1][2],x[1][1][3])))\
                         .map(lambda x: (x[1][0],x[1][1],x[1][2],x[1][3],1))\
                         .reduce(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))       

###calculate average for brief views
a1,v1,f1,s1,t1=brief_views.join(stats).map(lambda x: (x[0],(x[1][1][0],x[1][1][1],x[1][1][2],x[1][1][3])))\
                         .map(lambda x: (x[1][0],x[1][1],x[1][2],x[1][3],1))\
                         .reduce(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))       

#pass to lists
lis=[s,v,a,f]
lis1=[l/t for l in lis]

lis=[s1,v1,a1,f1]
lis2=[l/t1 for l in lis]

lis1.extend(lis2)

names=["vet_score","vet_views","vet_answers","vet_favorites","brief_score","brief_views","brief_answers","brief_favorites"]
identify_veterans_full = {n:l for n,l in zip(names,lis1)}

#### Checkpoints

In [ ]:
identify_veterans = {'brief_answers': 0.9705827067669173,
 'brief_favorites': 0.5757988721804511,
 'brief_score': 2.100892857142857,
 'brief_views': 553.4432330827068,
 'vet_answers': 1.2974161627267728,
 'vet_favorites': 1.3001649257833974,
 'vet_score': 3.54370533260033,
 'vet_views': 925.8889499725124}

In [ ]:
# identify_veterans = {
#     "vet_score": 0,
#     "vet_views": 0,
#     "vet_answers": 0,
#     "vet_favorites": 0,
#     "brief_score": 0,
#     "brief_views": 0,
#     "brief_answers": 0,
#     "brief_favorites": 0
# }

grader.score('spark__identify_veterans', identify_veterans)

## Identify veterans&mdash;full


Same as above, but on the full Stack Exchange data set.

No pre-parsed data is available for this question.


#### Checkpoints

* Total brief users: 1,848,628
* Total veteran users: 288,285

In [ ]:
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

posts_long = sc.textFile(localpath('spark-stack-data/allPosts'))
users_long = sc.textFile(localpath('spark-stack-data/allUsers'))

In [ ]:
def parsetime(x):
    if x is None:
        return None
    else:
        return datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%f')
    
def time_delta(x,y):
    if x is None or y is None:
        return None
    else:
        return (x-y).days
    
def user_parse(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line)
            if "Id" in root.attrib:
                uid = root.attrib['Id']
                cdate = root.attrib.get('CreationDate',None)
                return (int(uid),cdate)
            else:
                return ('Empty')
        except:
            return ('Empty')
    else:
        return ('Empty')
    
def date_post(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line)
            if "OwnerUserId" in root.attrib:
                uid = root.attrib['OwnerUserId']
                cdate = root.attrib.get('CreationDate',None)
                return (int(uid),cdate)
            else:
                return ('Empty')
        except:
            return ('Empty')
    else:
        return ('Empty')
    
def vet(x):
    if x is None:
        return 0
    if int(x) >= 100 and int(x) <= 150:
        return 1
    else:
        return 0
    
valid_users = users_long.map(user_parse).filter(lambda x: x != 'Empty').map(lambda x: (x[0],parsetime(x[1])))    

valid_posts = posts_long.map(date_post).filter(lambda x: x != 'Empty')\
                        .map(lambda x: (x[0],parsetime(x[1])))

all_user_info = valid_users.join(valid_posts)\
                           .map(lambda x: (x[0],time_delta(x[1][1],x[1][0])))         

all_i = all_user_info.map(lambda x: (x[0],vet(x[1]))).reduceByKey(lambda x,y: max(x,y))

veteran = all_i.filter(lambda x: x[1] == 1)
brief_views = all_i.filter(lambda x: x[1] == 0)

In [ ]:
def post_first_question_more(line):
    try:
        root = ET.fromstring(line)
        if "PostTypeId" and "OwnerUserId" and 'CreationDate' in root.attrib:
            uid = root.attrib['OwnerUserId']
            ptype = int(root.attrib['PostTypeId'])
            cdate = root.attrib['CreationDate']
            #a.attrib.get('Score',1)
            answer = root.attrib.get("AnswerCount",0)           
            view = root.attrib.get("ViewCount",0)           
            fav = root.attrib.get("FavoriteCount",0)            
            s = root.attrib.get("Score",0) 
            #filter out questions
            if ptype == 1:
                return (int(uid),cdate,int(answer),int(view),int(fav),int(s))
            else:
                return ('Empty')
        else:
            return ('Empty')
    except:
        return ('Empty')

#find out the first questions date
first_q= posts_long.map(post_first_question_more).filter(lambda x: x != 'Empty')\
              .map(lambda x: (x[0], parsetime(x[1])))\
              .reduceByKey(lambda x, y: min(x, y))\
              .map(lambda x: ((x[0],x[1]),1))

#find out all questions' performance
all_q = posts_long.map(post_first_question_more).filter(lambda x: x != 'Empty')\
              .map(lambda x: ((x[0],parsetime(x[1])),(x[2],x[3],x[4],x[5])))

#merge (acount, first_question_date) with performances
stats = first_q.join(all_q).map(lambda x: (x[0][0],(x[1][1][0],x[1][1][1],x[1][1][2],x[1][1][3])))

##calculate average for veterian
a,v,f,s,t=veteran.join(stats).map(lambda x: (x[0],(x[1][1][0],x[1][1][1],x[1][1][2],x[1][1][3])))\
                         .map(lambda x: (x[1][0],x[1][1],x[1][2],x[1][3],1))\
                         .reduce(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))       

###calculate average for brief views
a1,v1,f1,s1,t1=brief_views.join(stats).map(lambda x: (x[0],(x[1][1][0],x[1][1][1],x[1][1][2],x[1][1][3])))\
                         .map(lambda x: (x[1][0],x[1][1],x[1][2],x[1][3],1))\
                         .reduce(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))       

#pass to lists
lis=[s,v,a,f]
lis1=[l/t for l in lis]

lis=[s1,v1,a1,f1]
lis2=[l/t1 for l in lis]

lis1.extend(lis2)

In [ ]:
lis1 = [2.2561266751584585,
 1841.8189951718825,
 1.841804785404288,
 0.8661474978452715,
 1.129298016272707,
 1095.1468872554271,
 1.5033659787554736,
 0.3854817164205759]
names=["vet_score","vet_views","vet_answers","vet_favorites","brief_score","brief_views","brief_answers","brief_favorites"]
identify_veterans_full = {n:l for n,l in zip(names,lis1)}

In [ ]:
# identify_veterans_full = {
#     "vet_score": 0,
#     "vet_views": 0,
#     "vet_answers": 0,
#     "vet_favorites": 0,
#     "brief_score": 0,
#     "brief_views": 0,
#     "brief_answers": 0,
#     "brief_favorites": 0
# }

grader.score('spark__identify_veterans_full', identify_veterans_full)

## Word2vec


Word2Vec is an alternative approach for vectorizing text data. The vectorized representations of words in the vocabulary tend to be useful for predicting other words in the document, hence the famous example "vector('king') - vector('man') + vector('woman') ~= vector('queen')".

Let's see how good a Word2Vec model we can train using the **tags** of each Stack Exchange post as documents (this uses the full data set). Use the implementation of Word2Vec from Spark ML (this will require using DataFrames) to return a list of the top 25 closest synonyms to "ggplot2" and their similarity score in tuple format ("string", number).

The tags appear in the data as one string, you will need to separate them into individual tags. There is no need to further parse them beyond separating them.

In [ ]:
from pyspark.mllib.feature import Word2Vec
import re

def localpath(path):
    return  'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

def valid_xml(string):
    if '<row' in string  and '/>' in string :
        return 1
    else:
        return 0

def extracttags(string):
    try:
        tree = etree.fromstring(string)
        tags = tree.get("Tags")
        tags = tags[1:-1].split('><')
        return tags
    except: 
        return None

posts_complete = sc.textFile(localpath('spark-stack-data/allPosts/part-*.xml.gz'))
q8 = posts_complete.filter(valid_xml)\
    .map(extracttags)\
    .filter(lambda x:x is not None)
    
word2vec = Word2Vec()
model = word2vec.fit(q8)

synonyms = model.findSynonyms('ggplot2', 25)


In [ ]:
syn_list = [(k,v) for (k,v) in synonyms]

#### Parameters


The dimensionality of the vector space should be 100. The random seed should be 42 in `PySpark`.


#### Checkpoints

* Mean of the top 25 cosine similarities: 0.8012362027168274

In [ ]:
# word2vec = [("data.frame", 0.772650957107544)] * 25

grader.score('spark__word2vec', syn_list )

## Classification


We'd like to see if we can predict the tags of a **question** from its body text. Instead of predicting specific tags, we will instead try to predict if a question contains one of the top ten most common tags.  

To this end, we have separated out a train and a test set from the original data.  The training and tests sets were downloaded with the stats data at the beginning of the notebook.  You can also get them from S3:
  * `s3://dataincubator-course/spark-stats-data/posts_train.zip`
  * `s3://dataincubator-course/spark-stats-data/posts_test.zip`

This will involve two steps: first, find the ten most common tags for questions in the training data set (the tags have been removed from the test set). Then train a learner to predict from the text of the question (the `Body` attribute) if it should have one of those ten tags in it - you will need to process the question text with NLP techniques such as splitting the text into tokens.

Since we can't reliably pickle Spark models, instead return a list of your predictions, sorted by the question's `Id`.  This sorting is very important, as our grader expects the results to be submitted in a particular order. These predictions should be `0` if the question isn't expected to have a tag in the top ten, and `1` if it is.

As an example, if our top tags include `spark` and `python`, and we had the following questions:

```
<row Body="..." Id="1740" Tags="<machine-learning><spark><regression>" ... />
<row Body="..." Id="723" Tags="<statistics><neurons>" ... />
<row Body="..." Id="2740" Tags="<functional><python><spark><pyspark>" ... />
```

We would expect to return `[0, 1, 1]` (for the order `[723, 1740, 2740]`).  You may need to do some format manipulation in your DataFrame to get this.

#### Checkpoints

- Number of training posts with a tag in the top 10: `22525`
- Number without: `19540`

In [ ]:
from lxml import etree
from lxml.etree import XMLSyntaxError

def tag_post(line):
    if '<row' in line:
        try:
            root = etree.fromstring(line)
            if "Tags" in root.attrib:
                tag = root.attrib['Tags']
                return tag
            else:
                return ('Empty')
        except XMLSyntaxError:
            return ('Empty')
    else:
        return ('Empty')

In [ ]:
tag_posts = posts.map(tag_post).filter(lambda x: x!= 'Empty')

In [ ]:
import re

In [ ]:
train = sc.textFile(localpath('spark-stats-data/train'))

In [ ]:
def token_tag(string):
    tokens= [s for s in re.split("<|>", string) if s != '']
    token_count=[(t,1) for t in tokens]
    return token_count

In [ ]:
string = '<bayesian><prior><elicitation>'

In [ ]:
token_tag(string)

In [ ]:
common_tags = train.map(tag_post).filter(lambda x: x != 'Empty')\
                   .flatMap(token_tag).reduceByKey(lambda x,y: x+y)\
                   .takeOrdered(10, key=lambda x: -x[1])

In [ ]:
tags = [t[0] for t in common_tags]
tags

In [ ]:
def token_check(string):
    tokens= [s for s in re.split("<|>", string) if s != '']
    for t in tags:
        if t in tokens:
            return 1
    else:
        return 0

In [ ]:
string ='<regression><tag>'
token_check(string)

In [ ]:
p = re.compile('<p>(.+)</p>')

p = re.compile('>(.+)<')

In [ ]:
string = '<p>Last year, I read a blog post from <a href="http://anyall.org/">Brendan O\'Connor</a> entitled <a href="http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/">"Statistics vs. Machine Learning, fight!"</a> that discussed some of the differences between the two fields.  <a href="http://andrewgelman.com/2008/12/machine_learnin/">Andrew Gelman responded favorably to this</a>:</p>\n\n<p>Simon Blomberg: </p>\n\n<blockquote>\n  <p>From R\'s fortunes\n  package: To paraphrase provocatively,\n  \'machine learning is statistics minus\n  any checking of models and\n  assumptions\'.\n  -- Brian D. Ripley (about the difference between machine learning\n  and statistics) useR! 2004, Vienna\n  (May 2004) :-) Season\'s Greetings!</p>\n</blockquote>\n\n<p>Andrew Gelman:</p>\n\n<blockquote>\n  <p>In that case, maybe we should get rid\n  of checking of models and assumptions\n  more often. Then maybe we\'d be able to\n  solve some of the problems that the\n  machine learning people can solve but\n  we can\'t!</p>\n</blockquote>\n\n<p>There was also the <a href="http://projecteuclid.org/euclid.ss/1009213726"><strong>"Statistical Modeling: The Two Cultures"</strong> paper</a> by Leo Breiman in 2001 which argued that statisticians rely too heavily on data modeling, and that machine learning techniques are making progress by instead relying on the <em>predictive accuracy</em> of models.</p>\n\n<p>Has the statistics field changed over the last decade in response to these critiques?  Do the <em>two cultures</em> still exist or has statistics grown to embrace machine learning techniques such as neural networks and support vector machines?</p>\n'
''.join(p.findall(string))

In [ ]:
def format_body(string):
    p = re.compile('<p>(.+)</p>')
    lis = p.findall(string)
    return ''.join(p.findall(string))

In [ ]:
def tag_post(line):
    if '<row' in line:
        try:
            root = etree.fromstring(line)
            if 'Body' and "Tags" in root.attrib:
                body = root.attrib['Body']
                tag = root.attrib['Tags']
                return (body,token_check(tag))
            else:
                return ('Empty')
        except XMLSyntaxError:
            return ('Empty')
    else:
        return ('Empty')

In [ ]:

train.map(tag_post).filter(lambda x: x != 'Empty').take(5)

In [ ]:
train_set = train.map(tag_post).filter(lambda x: x != 'Empty').collect()

In [ ]:
train.map(tag_post).filter(lambda x: x != 'Empty').filter(lambda x: x[1]==0).count()

In [ ]:
from sklearn.pipeline import Pipeline
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


In [ ]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.feature import HashingTF, Tokenizer,RegexTokenizer
##save the computation from the traing process by cache()
training = sqlContext.createDataFrame(train_set, ["title", "label"]).cache()

In [ ]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml import Pipeline

tokenizer = RegexTokenizer(inputCol="title", outputCol="words", pattern="\\w")
tokenizer = Tokenizer(inputCol="title", outputCol="words")
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")

hashingTF = HashingTF(inputCol=remover.getOutputCol(), outputCol="features")
logreg = LogisticRegression(maxIter=1000000, regParam=0.7)

pipeline = Pipeline(stages=[tokenizer,remover, hashingTF, logreg])

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator


paramGrid = (ParamGridBuilder() 
    .addGrid(hashingTF.numFeatures, [2000])
    .addGrid(logreg.regParam, [10, 1, 0.2]) 
    .build())

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5)

In [ ]:
cvModel = crossval.fit(training)

In [ ]:
best_model = cvModel.bestModel

In [ ]:
test = sc.textFile(localpath('spark-stats-data/test'))

In [ ]:
def tag_post_test(line):
    if '<row' in line:
        try:
            root = etree.fromstring(line)
            if 'Body' and 'Id' and "PostTypeId" in root.attrib:
                if int(root.attrib['PostTypeId']) == 1:
                    pid = root.attrib['Id']
                    body = root.attrib['Body']
                    return (body, int(pid))
                else:
                    return ('Empty') 
            else:
                return ('Empty')
        except XMLSyntaxError:
            return ('Empty')
    else:
        return ('Empty')

In [ ]:
test.map(tag_post_test).filter(lambda x: x != 'Empty').count()

In [ ]:
test_set = test.map(tag_post_test).filter(lambda x: x != 'Empty')

In [ ]:
test_df = sqlContext.createDataFrame(test_set, ["title", "id"])

In [ ]:
test_df.show()

In [ ]:
better_prediction = cvModel.transform(test_df)

In [ ]:
better_prediction.show()

In [ ]:
selected = better_prediction.select("id", "prediction")

In [ ]:
for row in selected.collect()[5]:
    print(row)

In [ ]:
sort_selected = selected.sort("id").collect()

In [ ]:
predictions = [int(x['prediction']) for x in sort_selected]
len(predictions)

In [ ]:
# classification = [0] * 4649

grader.score('spark__classification', predictions)

## K-means (ungraded)


From your trained Word2Vec model, pass the vectors into a K-means clustering algorithm. Create a plot of the sum of squared error by calculating the square root of the sum of the squared distances for each point and its assigned cluster. For an independent variable use either the number of clusters k or the dimension of the Word2Vec vectorization.

*Copyright &copy; 2020 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*